In [ ]:
import specpy
import numpy as np
import json
from matplotlib import pyplot
import time

from skimage.feature import blob_dog

from pipeline2 import AcquisitionPipeline, PipelineLevels, DefaultNameHandler
from pipeline2.util import gen_json, dump_JSON
from pipeline2.imspector.imspector import get_current_stage_coords
from pipeline2.taskgeneration import ( SpiralOffsetGenerator, JSONFileConfigLoader, 
                                      DefaultStageOffsetsSettingsGenerator,
                                      AcquisitionTaskGenerator,
                                        DefaultLocationRemover, DefaultLocationKeeper,
                                     NewestDataSelector,
                             NewestSettingsSelector,
                             ZDCOffsetSettingsGenerator,)
from pipeline2.imspector import ImspectorConnection
from pipeline2.stoppingcriteria import TimedStoppingCriterion
from pipeline2.detection import ZDCSpotPairFinder



%matplotlib inline



In [ ]:
levels = PipelineLevels('overview', 'detail')

im = specpy.Imspector()
c = get_current_stage_coords(im)
sp = SpiralOffsetGenerator().withVerbose()
sp.withStart(c)
sp.withZOffset(c[2] + 5e-6)

atg = (AcquisitionTaskGenerator(levels.overview,
                                    DefaultLocationRemover(JSONFileConfigLoader(
                                        ['C:/Users/RESOLFT/Desktop/config_json/zdc_overview.json'])),
                                    DefaultStageOffsetsSettingsGenerator(sp))
           .withDelay(2.0))

pipeline = AcquisitionPipeline('1')

detector = ZDCSpotPairFinder(NewestDataSelector(pipeline, levels.overview), sigma=3, thresholds=[0.6, 0.6])
detector.withVerbose()
detector.withPlotDetections()


atg_detail = (AcquisitionTaskGenerator(levels.detail,
                                           DefaultLocationRemover(JSONFileConfigLoader(
                                               ['C:/Users/RESOLFT/Desktop/config_json/zdc_detail_xy.json',
                                               'C:/Users/RESOLFT/Desktop/config_json/zdc_detail_xz.json'], asMeasurements=False)),
                                           DefaultLocationKeeper(NewestSettingsSelector(pipeline, levels.overview)),
                                           ZDCOffsetSettingsGenerator(detector)
                                           )
            .withDelay(2.0))


pipeline.withPipelineLevels(levels)
pipeline.withNameHandler(DefaultNameHandler('C:/Users//RESOLFT/Desktop/TEST_GEN/', levels))
pipeline.withImspectorConnection(ImspectorConnection(im))

pipeline.withCallbackAtLevel(atg, levels.overview)
pipeline.withCallbackAtLevel(atg_detail, levels.overview)
pipeline.withAddedStoppingCondition(TimedStoppingCriterion(2400))

atg(pipeline)

In [ ]:
pipeline.run()

In [ ]:
pipeline.data

In [ ]:
for k,v in pipeline.data.items():
    pyplot.figure()
    pyplot.imshow(np.apply_along_axis(np.max, 2, v.data[0][0][0,:,:,:]))

In [ ]:
offOld = np.array([ -2.29999978e-06,   6.00000021e-07,   7.31280000e-06])
lenOld = np.array([  4.99500000e-05,   4.99500000e-05,   1.00000000e-05])
pszOld = np.array([  1.50000000e-07,   1.50000000e-07,   2.50000000e-07])
res = np.array([200.5, 135.0, 8.5])

offOld - lenOld / 2 + res * pszOld

## Title

asas $\sum_0^1 x$

In [ ]:
im = specpy.Imspector()

c = get_current_stage_coords(im)
print(c)
sp = SpiralOffsetGenerator()
sp.withStart(c)
sp.withZOffset(c[2] + 5e-6)



gen = DefaultStageOffsetsSettingsGenerator(sp)
gen = DefaultLocationKeeper(DefaultStageOffsetsSettingsGenerator(sp))

for _ in range(5):
    upd = gen()[0][0][0]
    print(upd)
    im.active_measurement().set_parameters('', upd)
    time.sleep(0.5)
    print(get_current_stage_coords(im))